In [7]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
# from astropy.io import fits
from ccdproc import ImageFileCollection
# Combiner, combine, subtract_dark, flat_correct
import matplotlib.pyplot as plt
%matplotlib inline

# filters

filters = ['g', 'r']
filter_full_names = ["Sloan g'", "Sloan r'"]

# subdirectories -- relative to observation and calibration directories

calibration_date = '2024-04-12'

calibration_directory = os.path.join(os.path.expanduser('~'), '2024 Sessions', calibration_date)

# the 30-second darks
dark_directory = os.path.join(calibration_directory, 'dark')

# the 1-second g and r flats
flat_directories_by_filter = {filter:os.path.join(calibration_directory, 'flat', filter) for filter in filters}

# the 1-second darks/biases
bias_directory = os.path.join(calibration_directory, 'bias')

# Trimmed image reader utility (needed because our images have four columns of NaNs)

def delete_last_columns(arr, columns_to_delete):
    column_count = np.shape(arr)[1]
    del_arr = np.delete(arr, slice(column_count - columns_to_delete, column_count), 1)
    return del_arr

def trimmed_image_reader(file):
    img = CCDData.read(file, unit=u.adu)
    data = img.data
    trimmed_data = delete_last_columns(data, 4)
    img.data = trimmed_data
    return img

dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')
flat_files_by_filter = {filter:ImageFileCollection(flat_directory).files_filtered(include_path='True')
               for filter:flat_directory in flat_directories_by_filter}
bias_files = ImageFileCollection(bias_directory).files_filtered(include_path='True')

darks = [trimmed_image_reader(file) for file in dark_files]
flats_by_filter = {filter:[trimmed_image_reader(file) for file in flat_files] for (filter, flat_files) in flat_files_by_filter}
biases = [trimmed_image_reader(file) for file in bias_files]

# Combine darks, flats, and biases

method = 'median'  # the method can be changed to 'average'

master_dark = combine(darks, method=method)
master_flat_by_filter = {filter:combine(flats, method=method) for (filter, flats) in flats_by_filter}
master_bias = combine(biases, method=method)



SyntaxError: invalid syntax (2584309456.py, line 47)